## WebCam
Accessing the webcam in a remote jupyter installation requires the WebRTC module to access the camera via the browser interface. Mkae sure to allow the browser to access your camera!

This method can also be used on a local jupyter instance.



In [1]:
# First step is to load all required modules
#from cv2 import VideoCapture, imshow, namedWindow, imwrite, destroyWindow, waitKey, cvtColor, COLOR_BGR2RGB, COLOR_BGRA2RGB
import matplotlib.pyplot as plt
import os

# for webrtc, we need
from ipywebrtc import CameraStream, ImageRecorder
# see https://ipywebrtc.readthedocs.io/en/latest/CameraStream.html

# and some image libraries, e.g. PILLOW (opencv should work as well)
import PIL.Image
import PIL.ImageFilter
import io
#from cv2 import VideoCapture, imshow, namedWindow, imwrite, destroyWindow, waitKey, cvtColor, COLOR_BGR2RGB, COLOR_BGRA2RGB


## Searching for cameras
You might have multiple cameras installed in your system and they might have different numbering. Try to search all of them (up to 3 in this example).


In [ ]:
# seperate cell here to open and display the camera

try:
    camera = CameraStream(constraints=
                      {'facing_mode': 'user',
                       'audio': False,
                       'video': { 'width': 640, 'height': 480 }
                       })
except:
    print("No Camera, nowhere ...")
    exit



In [ ]:
# we can display the camera preview here directly
camera


In [ ]:
# seperate cell here to show image recorder which allows to take a snapshot
image_recorder = ImageRecorder(stream=camera)


In [ ]:
image_recorder # opens the widget

In [ ]:
# make sure to have snapshot in the previous window!!!
# snapshot is available now, so use a new cell to fetch and save the image
rawimg = PIL.Image.open(io.BytesIO(image_recorder.image.value))


In [ ]:
# use a new cell to save the image
# the webcam image is in a "raw" format with alpha channel
# we could save it as .png, but we convert it to jpg right here
TARGET_DIR = "data"
# if the directory doesn't exist, we create it
if not TARGET_DIR in os.listdir():
    os.mkdir(TARGET_DIR)

file = os.sep.join([TARGET_DIR,"webcam.jpg"])

# convert to RGB => jpg
img = rawimg.convert('RGB')

img.save(file)



In [ ]:
# display image
img.show()


In [ ]:
# we can apply one of the Pillow filters to the image as well
img.filter(PIL.ImageFilter.BLUR)


In [ ]:
# we can also provess the image in the same style as before ...
from skimage import filters
from skimage.color import rgb2gray

fig, ax = plt.subplots(nrows=1, ncols=5, figsize=(10,2))
ax[0].imshow(img)
ax[0].set_title('RGB')

grayImg = rgb2gray(img)
    
ax[1].imshow(grayImg,cmap="Greys")
ax[1].set_title('Grayscale')

ax[2].hist(grayImg.ravel(), bins=256, histtype='step', color='black')
ax[2].set_title('Histogram')

thr = filters.threshold_otsu(grayImg)
print("Computed threshold: ",thr)
c = grayImg > thr
ax[3].imshow(c,cmap="Greys")
ax[3].set_title('Threshold')

d = filters.sobel(c)
ax[4].imshow(d,cmap="Greys")
ax[4].set_title('Edges')

for i,x in enumerate(ax):
    if i != 2:
        x.axis("off") 
plt.tight_layout()
plt.show()


In [ ]:
# finally, we can turn the image into a 3-dimension numpy array
import numpy as np
arImg = np.array(img)
# display the shape: height(number of rows)-width(number of columns)-colors
arImg.shape
# 

In [ ]:
# finally, we can turn the image into a 3-dimension numpy array
import numpy as np
arImg = np.array(img)
# display the shape: height(number of rows)-width(number of columns)-colors
arImg.shape
# now we modify individual image pixels, like putting a black square into 
# the middle of the image
for r in range(10):
    for c in range(10):
        arImg[arImg.shape[0]//2 + r,arImg.shape[1]//2 + c] = [0,0,0]

img2 = PIL.Image.fromarray(arImg)
img2.show()

## Summary
We have seen how to record an image from the webcam. You can deal with these images with a variety of libraries, like [pillow](https://pillow.readthedocs.io/en/stable/index.html), [scikit-image](https://scikit-image.org/docs/stable/) or [opencv](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_tutorials.html)
 
 Also, you can apply modifications at the pixel level with numpy and pillow.
 